In [1]:
from tensorflow.keras.models import Model
import tensorflow as tf
from model import lstm_resnet
import keras.backend as K
from helpers import f1
import warnings
import numpy as np
import matplotlib.pyplot as plt
from data_loader import DataGenerator
warnings.filterwarnings("ignore")

gpus = tf.config.list_physical_devices('GPU')
if gpus:
   tf.config.set_visible_devices(gpus[0], 'GPU')
   tf.config.experimental.set_memory_growth(gpus[0], True)

2025-07-26 17:44:18.173275: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-26 17:44:18.211555: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-26 17:44:18.211595: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-26 17:44:18.212741: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-26 17:44:18.218982: I tensorflow/core/platform/cpu_feature_guar

In [2]:
model = lstm_resnet(256, 256, 3, 15)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1])
model.summary()

2025-07-26 17:44:20.371200: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38435 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 15, 256, 256, 3)]    0         []                            
                                                                                                  
 time_distributed (TimeDist  (None, 15, 128, 128, 64)     9472      ['input_1[0][0]']             
 ributed)                                                                                         
                                                                                                  
 time_distributed_1 (TimeDi  (None, 15, 128, 128, 64)     256       ['time_distributed[0][0]']    
 stributed)                                                                                       
                                                                                              

In [3]:
train_gen = DataGenerator(
    image_dir="data_v3_processed/train/images/images",
    mask_dir="data_v3_processed/train/masks/masks",
    batch_size=8,
    time_steps=15,
    img_size=(256,256),
    normalize_images=True,
    normalize_masks=False,
    infinite=True,
)

In [4]:
val_gen = DataGenerator(
    image_dir="data_v3_processed/val/images/images",
    mask_dir="data_v3_processed/val/masks/masks",
    batch_size=8,
    time_steps=15,
    img_size=(256,256),
    normalize_images=True,
    normalize_masks=False,
)

In [5]:
history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=1,
                    batch_size=8,
                    steps_per_epoch=3000,
                    validation_steps=13)

2025-07-26 17:44:31.264793: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2025-07-26 17:44:38.292752: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fa397783f50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-26 17:44:38.293177: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX 6000 Ada Generation, Compute Capability 8.9
2025-07-26 17:44:38.298332: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1753551878.395189  159592 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3000/3000 [==============================] - 1246s 408ms/step - loss: 0.0095 - f1: 0.7595 - val_loss: 0.0129 - val_f1: 0.5810


In [6]:
model.save("lstm_resnet_model.h5")